# Getting the Camera's Rotation values
# Based on the Camera's and the Head's Locations

This notebook shows how to calculate the camera's rotation values based on the camera's and the head's locations
It's recommended to use our online visualizer to see the results: https://docs.datagen.tech/en/latest/_static/visualizer.html
The functionality below is based on our API (with minor changes it's possible to generalize it).

# Imports

In [1]:
import math
import numpy as np
from datagen.api import assets

# Inputs
Define the desired camera's and head's locations

In [2]:
CAM_X = -2.36
CAM_Y = -2.5
CAM_Z = -1.54

HEAD_X = 1.2
HEAD_Y = -0.85
HEAD_Z = -2.3

# Calculating the camera's rotation

This function calculates the camera's rotation based on its location and the head's location:
     · Since the origin (0, 0, 0) is located around the shoulder's level - we have a buffer at the camera's Z value.
     · Our camera is a pinhole camera. Therefore, the "roll" value doesn't affect the output and is set to 0.0.

In [3]:
def calc_cam_rotation(cam_location: assets.Point, head_location: assets.Point):
    # Our camera has a 0.12 buffer at the z value
    x = cam_location.x - head_location.x
    y = cam_location.y - head_location.y
    z = cam_location.z - 0.12 - head_location.z

    # Find the heading
    yaw_rad = math.atan2(x, y)
    yaw = 180 - ((yaw_rad * 180) / math.pi)
    if yaw > 180:
        yaw -= 360
    if yaw < -180:
        yaw += 360

    # Find the pitch
    pitch_rad = math.atan2(z, math.sqrt(x ** 2 + y ** 2))
    pitch = -((pitch_rad * 180) / math.pi)

    roll = 0.0  # It's a pinhole camera

    camera_rotation = assets.CameraRotation(yaw=yaw, pitch=pitch, roll=roll)

    return camera_rotation

In [4]:
cam_loc = assets.Point(x=CAM_X, y=CAM_Y, z=CAM_Z)
head_loc = assets.Point(x=HEAD_X, y=HEAD_Y, z=HEAD_Z)

cam_rot = calc_cam_rotation(cam_loc, head_loc)

print(f"Camera Yaw: {round(cam_rot.yaw, 2)} Degrees\nCamera Pitch: {round(cam_rot.pitch, 2)} Degrees\nCamera Roll: {cam_rot.roll} Degrees")

Camera Yaw: -65.13 Degrees
Camera Pitch: -9.26 Degrees
Camera Roll: 0.0 Degrees


# Getting a ring of n cameras around the head

In [5]:
num_of_cameras = 14
head_loc = assets.Point(x=HEAD_X, y=HEAD_Y, z=HEAD_Z)
radius = 1.8
z_value = 2.0
wavelength = "visible"  # Can be "nir" as well

This function returns a list of 'num_of_cameras' cameras which surround and point the head (evenly spaced at height = 'z_value')

In [6]:
def get_n_cameras_ring(num_of_cameras: int, head_location: assets.Point, radius: float, z_value: float, wavelength: str,
                       projection_type: assets.Projection = assets.Projection.PERSPECTIVE,
                       res_width: int = 512, res_height: int = 512,
                       fov_horiz: float = 9.0, fov_vert: float = 9.0,
                       sensor_width: float = 34):

    cameras = []

    if wavelength == "nir":
        wavelength = assets.Wavelength.NIR
    else:
        wavelength = assets.Wavelength.VISIBLE

    for i, xy_angle in enumerate(np.linspace(0, 360, num=num_of_cameras, endpoint=False)):
        xy_rad = (xy_angle * math.pi) / 180
        x = (math.sin(xy_rad) + head_location.x) * radius
        y = (math.cos(xy_rad) + head_location.y) * radius
        z = z_value + 0.12

        cam_location = assets.Point(x=x, y=y, z=z)
        cam_rotation = calc_cam_rotation(cam_location, head_location)

        camera = assets.Camera(
            name="Camera" + str(i),
            intrinsic_params=assets.IntrinsicParams(
                projection=projection_type,
                resolution_width=res_width,
                resolution_height=res_height,
                fov_horizontal=fov_horiz,
                fov_vertical=fov_vert,
                sensor_width=sensor_width,
                wavelength=wavelength
            ),
            extrinsic_params=assets.ExtrinsicParams(
                location=cam_location,
                rotation=cam_rotation
            )
        )

        cameras.append(camera)
        print(f"\n{camera.name}:")
        print(f"Camera location - X: {cam_location.x}, Y: {cam_location.y}, Z: {cam_location.z}")
        print(f"Camera rotation - Yaw: {cam_rotation.yaw} degrees, Pitch: {cam_rotation.pitch} degrees, Roll: {cam_rotation.roll} degrees")
    return cameras

In [7]:
cams = get_n_cameras_ring(num_of_cameras=num_of_cameras, head_location=head_loc, radius=radius, z_value=z_value, wavelength=wavelength)
len(cams)


Camera0:
Camera location - X: 2.16, Y: 0.2700000000000001, Z: 2.12
Camera rotation - Yaw: 139.3987053549955 degrees, Pitch: -71.06532415797712 degrees, Roll: 0.0 degrees

Camera1:
Camera location - X: 2.9409907304116043, Y: 0.0917439622243545, Z: 2.12
Camera rotation - Yaw: 118.4100268303194 degrees, Pitch: -65.28243606185931 degrees, Roll: 0.0 degrees

Camera2:
Camera location - X: 3.5672966684424536, Y: -0.4077183566542795, Z: 2.12
Camera rotation - Yaw: 100.58255716114203 degrees, Pitch: -60.7485567430627 degrees, Roll: 0.0 degrees

Camera3:
Camera location - X: 3.9148702419272827, Y: -1.129462318878634, Z: 2.12
Camera rotation - Yaw: 84.12280824299906 degrees, Pitch: -57.596647413403254 degrees, Roll: 0.0 degrees

Camera4:
Camera location - X: 3.9148702419272827, Y: -1.930537681121366, Z: 2.12
Camera rotation - Yaw: 68.2970923481266 degrees, Pitch: -55.80252767672689 degrees, Roll: 0.0 degrees

Camera5:
Camera location - X: 3.5672966684424536, Y: -2.652281643345721, Z: 2.12
Camera

14